In [ ]:
import os

In [ ]:
from antirouge import config
from antirouge.data import load_data_generators

In [ ]:
# Optional do not use GPU
# must be called before importing tensorflow
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Preprocessing

Sentence embedding is pretty expensive. Thus we're embedding the texts to vectors beforehand to avoid doing it in each epoch.

In [ ]:
from antirouge.pre import pre_embed_tsv_folder

In [ ]:
pre_embed_tsv_folder('pre/cnn_dailymail_add', 'USE', batch_size=1000)

In [ ]:
pre_embed_tsv_folder('pre/billsum_cross', 'InferSent', batch_size=1000)

In [ ]:
# Doing for a bunch of settings
for embed_method in ['USE', 'USE-Large', 'InferSent']:
    for folder in ['pre/cnn_dailymail_add', 
                   'pre/cnn_dailymail_cross',
                   'pre/cnn_dailymail_delete',
                   'pre/cnn_dailymail_replace']:
        pre_embed_tsv_folder(folder, embed_method, batch_size=1000)

# Sentence Embedding Experiment

## Load Data

In [ ]:
from antirouge.data import load_data_generators

In [ ]:
# USE embedded input
train_folder = os.path.join('pre/cnn_dailymail_add', 'train.tsv_embed', 'USE')
test_folder = os.path.join('pre/cnn_dailymail_add', 'test.tsv_embed', 'USE')
validation_folder = os.path.join('pre/cnn_dailymail_add', 'validation.tsv_embed', 'USE')

In [ ]:
data_iters = load_data_generators(train_folder, validation_folder, test_folder, bsize=8)

## Run models

In [ ]:
from antirouge.model import train_model, create_FC_model, create_LSTM_model, create_CNN1D_model

In [ ]:
model = create_FC_model(512)

In [ ]:
model = create_LSTM_model(512)

In [ ]:
model = create_CNN1D_model(512)

In [ ]:
train_model(model, data_iters)

# Experiments

In [ ]:
for embed_method in ['USE', 'USE-Large', 'InferSent']:
    for folder in ['pre/cnn_dailymail_add', 
                   'pre/cnn_dailymail_cross',
                   'pre/cnn_dailymail_delete',
                   'pre/cnn_dailymail_replace']:
        # FIXME assuming preprocessed data exists
        train_folder = os.path.join(folder, 'train.tsv_embed', embed_method)
        test_folder = os.path.join(folder, 'test.tsv_embed', embed_method)
        validation_folder = os.path.join(folder, 'validation.tsv_embed', embed_method)
        data_iters = load_data_generators(train_folder, validation_folder, test_folder, bsize=8)
        for model_fn in [create_FC_model, create_LSTM_model, create_CNN1D_model]:
            model = model_fn(4096 if embed_method == 'InferSent' else 512)
            # TODO save the log somewhere
            train_model(model, data_iters)